In [ ]:
from bs4 import BeautifulSoup
import re, time

import urllib.request

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

def getTextFromArticle(link):
    try:
        opener = AppURLopener()
        response = opener.open(link).read()
        soup = BeautifulSoup(response, "lxml")
        dataHtml = soup.find("div", {"itemprop": "articleBody"})

        #print(dataHtml.getText())
        text=dataHtml.getText()
        #print(type(text))

        #Remove links
        regex=r'https([^ ]+)'
        t=re.sub(regex, '', text)
        #print(t)
        #Remove ads
        ad_list=[r'Mainos\s\(Teksti jatkuu alla\)',
                 r'try {',
                 r"sanomaAd.+",
                 r'catch\s\(err\).*',
                 r'try+',
                 r'{',
                 r'}',
                 r'\s\s\s\s\s\s\s\s\s\s\s\s\s\s']
        for word in ad_list:
            #print(word)
            t=re.sub(word, '', t)

        text=t.replace("Mainos päättyy","")

        processed_text=re.sub(r'\s\s', ' ', text)
        #print(processed_text)
        return processed_text
    except:
        print("Could not get text from article: ",link)
        return "Error"

link="https://www.hs.fi/urheilu/art-2000005974075.html"
getTextFromArticle(link)
print(getTextFromArticle(link))

In [ ]:
def getDataFromLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("div")
    
    #links = [a.find("a") for a in dataHtml]
    #print(links)
    #teasers=soup.find_all("h2")
    teasers=soup.findAll("article", {"class": "teaser full-width fast"})
    data=[]
    for t in teasers:
        link="https://hs.fi"+str(t.find("a")['href'])
        teaser=str(t.find("h2")).replace("<h2>","").replace("</h2>","")
        teaser=re.sub("<span.*?span>", "", teaser)
        article_text=getTextFromArticle(link)
        #print(link)
        #print(teaser)
        #getTextFromLink(link)
        data.append([link,teaser,article_text])
        #break
    #data = dataHtml.find("li", {"class": "bold"})
    #print(teasers[0])
    #print(soup)
    #print(rawList)
    return data

import csv
def writeToCsv(file,data):
    with open(file,'a') as f:
        wr = csv.writer(f, dialect='excel')
        wr.writerows(data)

link='https://hs.fi/'
data=getDataFromLink(link)
for d in data:
    print(d)
#writeToCsv("hsData.csv", data)

In [ ]:
import datetime

dir_name="./hs_data/"
time=str(datetime.datetime.now().date())
filename=time + " hs data.csv"
f=open(dir_name+filename,'a')
f.close()

raw_link='https://www.hs.fi'
print("Writing to file "+dir_name+filename)
failed=0

link=raw_link
data=getDataFromLink(link)
writeToCsv(dir_name+filename, data)
    
print("Finished")